In [ ]:
# Add this at the top
import sys
sys.path.append('./source')

from source.feature_selection import select_top_features
from source.propensity_model import train_propensity_model
from source.weighting import compute_iptw
from source.balance import calculate_smd
from source.causal_estimation import estimate_ate

import pandas as pd
import numpy as np

# Simulate data
np.random.seed(42)
df = pd.DataFrame({
    'product_rating': np.random.normal(4, 0.5, 1000),
    'price_index': np.random.normal(1, 0.2, 1000),
    'page_views': np.random.randint(1, 20, 1000),
    'fulfilled_by_platform': np.random.randint(0, 2, 1000),
    'customer_cohort': np.random.choice([0, 1], 1000),
    'treatment': np.random.randint(0, 2, 1000),
    'converted': np.random.randint(0, 2, 1000)
})

X = df.drop(columns=['treatment', 'converted'])
y_treatment = df['treatment']
y_outcome = df['converted']

# Feature selection
top_features = select_top_features(X, y_treatment)
X_selected = X[top_features]

# Propensity model
model, p_scores = train_propensity_model(X_selected, y_treatment)

# IPTW weighting
weights = compute_iptw(y_treatment, p_scores)

# Balance check
smd = calculate_smd(X_selected, y_treatment, weights)
print("SMD after weighting:", smd)

# ATE estimation
ate = estimate_ate(y_outcome, y_treatment, weights)
print("Estimated ATE:", ate)